In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import pandas

seed = 7
np.random.seed(seed)
def string2bits(s=''):
    return [bin(ord(x))[2:].zfill(8) for x in s]
def bits2string(b=None):
    return ''.join([chr(int(x, 2)) for x in b])
dataframe = pandas.read_csv("hash_md5_dataset_sample.csv", header=None)
dataset = dataframe.values
x = dataset[:, 0:2]
X = []
Y= []
x_test=[]
y_test=[]

for i in range(400000):
    X.append(((''.join(string2bits(x[i][0])))))

for i in range(400000):
    Y.append(((''.join(string2bits(x[i][1])))))

for i in range(400000,500000):
    x_test.append(((''.join(string2bits(x[i][0])))))

for i in range(400000,500000):
    y_test.append(((''.join(string2bits(x[i][1])))))


x_train = np.array([[int(x) for x in y] for y in X])
y_train = np.array([[int(x) for x in y] for y in Y])

x_test = np.array([[int(x) for x in y] for y in x_test])
y_test = np.array([[int(x) for x in y] for y in y_test])

model = Sequential()
model.add(Dense(256, input_dim=256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(160,activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()
model.fit(x_train, y_train,
          epochs=200,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=1024, verbose=1)
classes = model.predict(x_test, batch_size=128)
print(score)

In [44]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss
out = model.predict_proba(x_test)
out = np.array(out)

threshold = np.arange(0.1,0.9,0.1)
acc = []
accuracies = []
best_threshold = np.zeros(out.shape[1])
for i in range(out.shape[1]):
    y_prob = np.array(out[:,i])
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        acc.append( matthews_corrcoef(y_test[:,i],y_pred))
    acc   = np.array(acc)
    index = np.where(acc==acc.max()) 
    accuracies.append(acc.max()) 
    best_threshold[i] = threshold[index[0][0]]
    acc = []

y_pred = np.array([[1 if out[i,j]>=best_threshold[j] else 0 for j in range(y_test.shape[1])] for i in range(len(y_test))])
y_pred
y_test

  32/1000 [..............................] - ETA: 0s

array([[0, 1, 1, ..., 0, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 0, 1],
       ..., 
       [0, 1, 1, ..., 0, 1, 0],
       [0, 1, 1, ..., 0, 0, 1],
       [0, 1, 0, ..., 1, 0, 1]])

In [45]:
print hamming_loss(y_test,y_pred)
total_correctly_predicted = len([i for i in range(len(y_test)) if (y_test[i]==y_pred[i]).sum() == 160])
print total_correctly_predicted/400. #exact accuracy for eg y_pred = [0,0,1,1,1] and y_test=[0,0,1,1,1]
print total_correctly_predicted

0.3877375
0.0
0
